In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path 
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [2]:
from pathlib import Path
DATA_DIR = Path("asl_alphabet_train/asl_alphabet_train")
subdirs = [p.name for p in DATA_DIR.iterdir() if p.is_dir()]
print(f"Subcarpetas encontradas: {len(subdirs)} ⇒ {subdirs[:10]}...")

Subcarpetas encontradas: 29 ⇒ ['R', 'U', 'I', 'N', 'G', 'Z', 'T', 'S', 'A', 'F']...


In [3]:
def load_data(img_size=64, limit_per_class=None):
    X, y = [], []
    for idx, category in enumerate(CATEGORIES):
        folder = Path(DATA_DIR) / category
        imgs = sorted(folder.iterdir())
        if limit_per_class:
            imgs = imgs[:limit_per_class]          # para debug rápido
        for img_path in imgs:
            img = cv2.imread(str(img_path))
            if img is None:                # <-- imagen corrupta o mal leída
                print(f"[WARN] Saltando {img_path}")
                continue
            img = cv2.resize(img, (img_size, img_size), cv2.INTER_AREA)
            X.append(img)
            y.append(idx)
    X = np.asarray(X, dtype="float32") / 255.0
    y = to_categorical(y, num_classes=len(CATEGORIES))
    return X, y


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 64
BATCH = 32
SEED = 42

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # <<<<<< ESTA LÍNEA ES CLAVE
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,  # Si tiene sentido
    fill_mode='nearest'
)


train_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH,
    class_mode='categorical',
    subset='training',
    seed=SEED
)

val_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH,
    class_mode='categorical',
    subset='validation',
    seed=SEED
)



Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


In [9]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras.optimizers import Adam

IMG_SHAPE = (64, 64, 3)
N_CLASSES = 29
LR = 1e-3
EPOCHS = 10      # pon 10‑15 para el barrido inicial

def build_baseline():
    x_in = Input(shape=IMG_SHAPE)
    x = Conv2D(32,(3,3),activation='relu')(x_in); x = MaxPooling2D()(x)
    x = Conv2D(64,(3,3),activation='relu')(x); x = MaxPooling2D()(x)
    x = Flatten()(x)
    x = Dense(128,activation='relu')(x); x = Dropout(0.3)(x)
    out = Dense(N_CLASSES, activation='softmax')(x)
    return Model(x_in, out)

def build_mobilenet():
    base = MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
    base.trainable = False          # primer round: solo cabeza
    x = Flatten()(base.output)
    x = Dense(256,activation='relu')(x)
    out = Dense(N_CLASSES, activation='softmax')(x)
    return Model(base.input, out)

def build_efficientnet():
    base = EfficientNetB0(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
    base.trainable = False
    x = Flatten()(base.output)
    x = Dense(256,activation='relu')(x)
    out = Dense(N_CLASSES, activation='softmax')(x)
    return Model(base.input, out)

models_to_try = {
    "baseline": build_baseline,
    "mobilenet": build_mobilenet,
    "efficientnet": build_efficientnet
}

results = {}
for name, builder in models_to_try.items():
    print(f"\n🔧 Entrenando {name} ...")
    model = builder()
    model.compile(optimizer=Adam(LR), loss='categorical_crossentropy', metrics=['accuracy'])
    h = model.fit(train_gen,
                  epochs=EPOCHS,
                  validation_data=val_gen,
                  callbacks=[],
                  verbose=1)
    best_val = max(h.history['val_accuracy'])
    results[name] = best_val
    model.save(f"{name}.keras")
    print(f"➡️  {name}: best val_accuracy = {best_val:.4f}")

print("\n=== RESUMEN ===")
for n,a in results.items():
    print(f"{n:<12} : {a:.4f}")



🔧 Entrenando baseline ...
Epoch 1/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 84s 39ms/step - accuracy: 0.1441 - loss: 2.9596 - val_accuracy: 0.2500 - val_loss: 2.3899
Epoch 2/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 82s 38ms/step - accuracy: 0.4183 - loss: 1.8408 - val_accuracy: 0.3855 - val_loss: 1.8900
Epoch 3/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 84s 39ms/step - accuracy: 0.5292 - loss: 1.4471 - val_accuracy: 0.4806 - val_loss: 1.6049
Epoch 4/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 84s 39ms/step - accuracy: 0.5937 - loss: 1.2177 - val_accuracy: 0.5233 - val_loss: 1.4140
Epoch 5/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 82s 38ms/step - accuracy: 0.6414 - loss: 1.0703 - val_accuracy: 0.5348 - val_loss: 1.3882
Epoch 6/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 82s 38ms/step - accuracy: 0.6755 - loss: 0.9727 - val_accuracy: 0.5589 - val_loss: 1.3195
Epoch 7/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 84s 39ms/step - accuracy: 0.7009 - loss: 0.8884 - val_accuracy: 0.5626 - val_loss: 1.3069
Epoch 8/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 85s 39m

/var/folders/nb/c0sg3gpn54s2yvl29bhbcx9h0000gn/T/ipykernel_10384/969221915.py:21: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')


Epoch 1/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 79s 36ms/step - accuracy: 0.4333 - loss: 1.9447 - val_accuracy: 0.4010 - val_loss: 2.1429
Epoch 2/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 86s 39ms/step - accuracy: 0.5994 - loss: 1.2886 - val_accuracy: 0.4353 - val_loss: 2.0579
Epoch 3/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 170s 78ms/step - accuracy: 0.6398 - loss: 1.1673 - val_accuracy: 0.4415 - val_loss: 2.0170
Epoch 4/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 168s 77ms/step - accuracy: 0.6541 - loss: 1.1021 - val_accuracy: 0.4458 - val_loss: 2.0777
Epoch 5/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 170s 78ms/step - accuracy: 0.6725 - loss: 1.0451 - val_accuracy: 0.4644 - val_loss: 2.0213
Epoch 6/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 204s 94ms/step - accuracy: 0.6848 - loss: 1.0087 - val_accuracy: 0.4600 - val_loss: 2.0562
Epoch 7/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 239s 110ms/step - accuracy: 0.6895 - loss: 0.9892 - val_accuracy: 0.4643 - val_loss: 2.0467
Epoch 8/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 211s 97ms/step - accuracy: 0

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

IMG_SHAPE = (64, 64, 3)
N_CLASSES = 29  # Cambia si tienes más/menos letras

# Modelo base preentrenado
base_model = MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Congelar capas preentrenadas

# Añadir nueva cabeza de clasificación
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
output = Dense(N_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compilar
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Entrenar
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)


In [ ]:
# Desbloquear parte de MobileNet
base_model.trainable = True

# Opción: solo afinar las últimas N capas
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Recompilar con menor LR
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=2)
]

# Fine-tune
history_ft = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=callbacks
)


In [ ]:
loss, acc = model.evaluate(test_gen)
print(f"🔍 Accuracy final en test: {acc:.4f}")


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

model = load_model("efficientnet.keras")

# ⚠️ Descongelar las últimas N capas del backbone
UNFREEZE = 30
for layer in model.layers[-UNFREEZE:]:
    if not isinstance(layer, Dense):   # evita re‑compilar las densas
        layer.trainable = True

model.compile(optimizer=Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    ReduceLROnPlateau(factor=0.2, patience=3, verbose=1),
    EarlyStopping(patience=6, restore_best_weights=True, verbose=1)
]

history_ft = model.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen,
    callbacks=callbacks
)
model.save("asl_best_finetuned.keras")


In [ ]:
test_gen = ImageDataGenerator(rescale=1/255).flow_from_directory(
    "asl_alphabet_test",
    target_size=(64,64),
    batch_size=32,
    shuffle=False,
    class_mode='categorical'
)

loss, acc = model.evaluate(test_gen)
print(f"📊 Accuracy test: {acc:.4f}")

In [ ]:
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title("Precisión del modelo")
plt.xlabel("Época")
plt.ylabel("Precisión")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_true = test_gen.classes
y_pred = model.predict(test_gen, verbose=0).argmax(axis=1)
print(classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys()))

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("efficientnet.keras")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1/255)

test_gen = test_datagen.flow_from_directory(
    "asl_alphabet_test",
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

loss, acc = model.evaluate(test_gen)
print(f"📊 Accuracy en test: {acc:.4f}")

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_true = test_gen.classes
y_pred = model.predict(test_gen, verbose=1).argmax(axis=1)

print(classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys()))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=False, cmap='Blues')
plt.title("Matriz de Confusión")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# ===== Mostrar máximos =====
best_epoch = int(np.argmax(history.history['val_accuracy']))
best_val  = history.history['val_accuracy'][best_epoch]
best_train = history.history['accuracy'][best_epoch]
print(f"Mejor época: {best_epoch+1}  |  "
      f"accuracy train: {best_train:.4f}  |  accuracy val: {best_val:.4f}")

# ===== Graficar curva =====
pd.DataFrame(history.history)[['accuracy','val_accuracy']].plot()
plt.title("Evolución de accuracy"); plt.xlabel("Época"); plt.ylabel("Accuracy"); plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# --- Graficar Accuracy ---
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title("Precisión (Accuracy)")
plt.xlabel("Época")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)

# --- Graficar Loss ---
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title("Pérdida (Loss)")
plt.xlabel("Época")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
history_ft.history['accuracy']  # o 'val_accuracy', etc.
